In [1]:
import pandas as pd

In [2]:
path_trans = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_transactions.csv")
path_users = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_users.csv")
path_products = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_products.csv")

In [3]:
df_products = pd.read_csv(path_products, encoding = 'utf-8')
df_users = pd.read_csv(path_users, encoding = 'utf-8')
df_trans = pd.read_csv(path_trans, encoding = 'utf-8' )


In [4]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          13495 non-null  int64  
 1   order_id         13495 non-null  int64  
 2   Date             13495 non-null  object 
 3   Amount           13495 non-null  float64
 4   Merchant_id      13495 non-null  int64  
 5   Purchase_status  13495 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 632.7+ KB


In [8]:
df_trans = df_trans.merge(
    df_products, on="Merchant_id", how="left"
)

df_trans["Revenue"] = df_trans["Amount"] * df_trans["Rate_pct"]

In [9]:
jan_2020_revenue = df_trans.loc[
    (df_trans['Date'] >= "2020-01-01") &
    (df_trans['Date'] < "2021-01-01"), 
    "Revenue"].sum()


In [10]:
print(jan_2020_revenue)

1871272702.0


In [5]:
df_trans['Date'] = pd.to_datetime(df_trans['Date'])


In [ ]:
most_profitable_month = (
    df_trans
    .groupby(df_trans["Date"].dt.to_period("M"))["Revenue"]
    .sum()
    .idxmax()
)
total_for_month = (
    df_trans
    .groupby(df_trans['Date'].dt.to_period("M"))["Revenue"]
    .sum()
    .max()
)
print(most_profitable_month,total_for_month)

2020-09 170220000.0


In [21]:
weekday_avg = (
    df_trans
    .groupby(df_trans["Date"].dt.day_name())["Revenue"]
    .mean()
)
most_profitable_day = weekday_avg.idxmax()
least_profitable_day = weekday_avg.idxmin()


print(most_profitable_day, least_profitable_day)

Wednesday Monday
